### priority queue(우선순위 큐)
선입 선출이 아닌 우선순위의 순서대로 삭제되는 큐

In [3]:
from exceptions import Empty

class PriorityQueueBase:
    class _Item:
        __slots__ = '_key', '_value'

        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __lt__(self, other):
            return self._key < other._key

        def __repr__(self):
            return '({0},{1})'.format(self._key, self._value)

    def is_empty(self):
        return len(self) == 0

    def __len__(self):
        raise NotImplementedError('must be implemented by subclass')

    def add(self, key, value):
        raise NotImplementedError('must be implemented by subclass')

    def _min(self):
        raise NotImplementedError('must be implemented by subclass')

    def remove_min(self):
        raise NotImplementedError('must be implemented by subclass')

### heap
최소 트리 : 트리의 모든 노드에 대ㅐㅎ 노드의 데이터 값이 자식 노드의 데이터 값보다 작거나 같은것
최소 히프 : 최소 트리이면서 완전 이진 트리인 것.
우선순위로 삽입 삭제시
히프는 삽입, 삭제 둘 다 O(log2(n))으로 가능하다

In [4]:
class HeapPriorityQueue(PriorityQueueBase): #최소 heap
    def _parent(self, j):
        return (j-1) // 2

    def _left(self, j):
        return 2*j + 1
  
    def _right(self, j):
        return 2*j + 2

    def _has_left(self, j):
        return self._left(j) < len(self._data)
  
    def _has_right(self, j):
        return self._right(j) < len(self._data)
  
    def _swap(self, i, j):
        self._data[i], self._data[j] = self._data[j], self._data[i]

    def _upheap(self, j):
        parent = self._parent(j)
        if j > 0 and self._data[j] < self._data[parent]:
            self._swap(j, parent)
            self._upheap(parent)
  
    def _downheap(self, j):
        if self._has_left(j):
            left = self._left(j)
            small_child = left
            if self._has_right(j):
                right = self._right(j)
                if self._data[right] < self._data[left]:
                    small_child = right
            if self._data[small_child] < self._data[j]:
                self._swap(j, small_child)
                self._downheap(small_child)

    def __init__(self):
        self._data = []

    def __len__(self):
        return len(self._data)

    def add(self, key, value):
        self._data.append(self._Item(key, value))
        self._upheap(len(self._data) - 1)
  
    def _min(self):
        if self.is_empty():
            raise Empty('Priority queue is empty.')
        item = self._data[0]
        return (item._key, item._value)

    def remove_min(self):
        if self.is_empty():
            raise Empty('Priority queue is empty.')
        self._swap(0, len(self._data) - 1)
        item = self._data.pop()
        self._downheap(0)
        return (item._key, item._value)

### map
맵은 key와 value의 쌍을 저장하고 key를 통해 검색, 삽입, 삭제 등을 한다. key의 값은 유일하다.  
파이썬에서는 dictionary가 map의 역할을 한다.  


In [7]:
from collections import MutableMapping

In [8]:
class MapBase(MutableMapping): #MutableMapping의 하위클래스
    class _Item:
        __slots__ = '_key', '_value'

        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __eq__(self, other):
            return self._key == other._key

        def __ne__(self, other):
            return not (self == other)

        def __lt__(self, other):               
            return self._key < other._key

### Binary Search Tree
히프의 문제점 : 삭제시 O(n), 특정 노드 검색 O(n)  
이진 검색 트리는 트리 내에서 특정 데이터를 효율적으로 찾도록 하는 트리  
모든 노드는 유일한 키 값을 가지며 왼쪽 서브 트리에 저장된 키값들은 루트노드보다 작고  
오른쪽 서브트리에 저장된 키 값들은 루트보다 크다. 서브트리들도 이진 검색트리.
LinkedBinaryTree와 MapBase가 필요하다

In [13]:
from linked_binary_tree import LinkedBinaryTree
#from map_base import MapBase

class TreeMap(LinkedBinaryTree, MapBase):
    class Position(LinkedBinaryTree.Position):
        def key(self):
            return self.element()._key

        def value(self):
            return self.element()._value

    def _subtree_search(self, p, k): #검색 메소드. 검색하는 값이 없으면 None이 아닌 마지막 리프노드가 return 된다
        if k == p.key():
            return p                                         
        elif k < p.key():
            if self.left(p) is not None:
                return self._subtree_search(self.left(p), k)   
        else:
            if self.right(p) is not None:
                return self._subtree_search(self.right(p), k)
        return p

    def _subtree_first_position(self, p): #가장 작은 값 리턴하는 메소드
        walk = p
        while self.left(walk) is not None:
            walk = self.left(walk)
        return walk

    def _subtree_last_position(self, p): #가장 큰 값 리턴하는 메소드
        walk = p
        while self.right(walk) is not None:
            walk = self.right(walk)
        return walk
  
    def first(self):
        return self._subtree_first_position(self.root()) if len(self) > 0 else None

    def last(self):
        return self._subtree_last_position(self.root()) if len(self) > 0 else None

    def before(self, p): #p의 바로 앞의 값을 찾는 메소드
        self._validate(p)
        if self.left(p): #왼쪽이 있으면 왼쪽 서브트리의 가장 큰값.
            return self._subtree_last_position(self.left(p))
        else: #왼쪽이 없으때 자신이 부모의 오른쪽 노드이면 부모가 바로 앞의 값.
            walk = p
            above = self.parent(walk)
            while above is not None and walk == self.left(above): #자신이 부모의 왼쪽 노드이면 부모노드로 계속 올라감.
                walk = above
                above = self.parent(walk)
            return above#그렇게 부모노드로 계속 올라가다 오른쪽에 달리게 되면 그때의 노드의 부모노드가 앞의 값

    def after(self, p): #p의 바로 뒤의 값을 찾는 메소드(before의 반대)
        self._validate(p)
        if self.right(p):
            return self._subtree_first_position(self.right(p))
        else:
            walk = p
            above = self.parent(walk)
            while above is not None and walk == self.right(above):
                walk = above
                above = self.parent(walk)
            return above

    def find_position(self, k): #k의 위치 찾는 메소드
        if self.is_empty():
            return None
        else:
            p = self._subtree_search(self.root(), k)
            self._rebalance_access(p)
            return p

    def delete(self, p): #삭제 메소드
        self._validate(p)
        if self.left(p) and self.right(p): #왼쪽 오른쪽 둘다 있으면 삭제할 노드의 전 값을 찾아
            replacement = self._subtree_last_position(self.left(p))
            self._replace(p, replacement.element()) #교환한다. 삭제할 노드의 전 값은 리프노드거나 오른쪽 노드가 없으므로
            p =  replacement
        parent = self.parent(p)
        self._delete(p) #LinkedBinaryTree의 _delete사용가능
        self._rebalance_delete(parent)
      
    def __getitem__(self, k): #k가 key값인거 찾는 메소드
        if self.is_empty():
            raise KeyError('Key Error: ' + repr(k))
        else:
            p = self._subtree_search(self.root(), k)
            self._rebalance_access(p)
            if k != p.key():
                raise KeyError('Key Error: ' + repr(k))
            return p.value()

    def __setitem__(self, k, v): #k가 key인것의 element를 v로 바꾸는 메소드. k가 없으면 새로운 노드를 만든다.
        if self.is_empty():
            leaf = self._add_root(self._Item(k,v))
        else:
            p = self._subtree_search(self.root(), k)
            if p.key() == k:
                p.element()._value = v
                self._rebalance_access(p)
                return
            else:
                item = self._Item(k,v)
                if p.key() < k:
                    leaf = self._add_right(p, item)
                else:
                    leaf = self._add_left(p, item)
        self._rebalance_insert(leaf)

    def __delitem__(self, k):
        if not self.is_empty():
            p = self._subtree_search(self.root(), k)
            if k == p.key():
                self.delete(p)
                return
            self._rebalance_access(p)
        raise KeyError('Key Error: ' + repr(k))

    def __iter__(self): #iter는 처음거 찾아서 다음거 계속해서 넘겨준다.
        p = self.first()
        while p is not None:
            yield p.key()
            p = self.after(p)

    def __reversed__(self):
        p = self.last()
        while p is not None:
            yield p.key()
            p = self.before(p)

    def find_min(self):
        if self.is_empty():
            return None
        else:
            p = self.first()
            return (p.key(), p.value())

    def find_max(self):
        if self.is_empty():
            return None
        else:
            p = self.last()
            return (p.key(), p.value())

    def find_le(self, k):
        if self.is_empty():
            return None
        else:
            p = self.find_position(k)
            if k < p.key():
                p = self.before(p)
            return (p.key(), p.value()) if p is not None else None

    def find_lt(self, k):
        if self.is_empty():
            return None
        else:
            p = self.find_position(k)
            if not p.key() < k:
                p = self.before(p)
            return (p.key(), p.value()) if p is not None else None

    def find_ge(self, k):
        if self.is_empty():
            return None
        else:
            p = self.find_position(k)
            if p.key() < k:
                p = self.after(p)
            return (p.key(), p.value()) if p is not None else None

    def find_gt(self, k):
        if self.is_empty():
            return None
        else:
            p = self.find_position(k)
            if not k < p.key():                   
                p = self.after(p)
            return (p.key(), p.value()) if p is not None else None
  
    def find_range(self, start, stop):
        if not self.is_empty():
            if start is None:
                p = self.first()
            else:
                p = self.find_position(start)
                if p.key() < start:
                    p = self.after(p)
            while p is not None and (stop is None or p.key() < stop):
                yield (p.key(), p.value())
                p = self.after(p)

    def _rebalance_insert(self, p):
        pass

    def _rebalance_delete(self, p):
        pass

    def _rebalance_access(self, p):
        pass

    def _relink(self, parent, child, make_left_child):
        if make_left_child:
            parent._left = child
        else:
            parent._right = child
        if child is not None:
            child._parent = parent

    def _rotate(self, p):
        x = p._node
        y = x._parent
        z = y._parent
        if z is None:            
            self._root = x
            x._parent = None        
        else:
            self._relink(z, x, y == z._left)
        if x == y._left:
            self._relink(y, x._right, True)
            self._relink(x, y, False)
        else:
            self._relink(y, x._left, False)
            self._relink(x, y, True)

    def _restructure(self, x):
        y = self.parent(x)
        z = self.parent(y)
        if (x == self.right(y)) == (y == self.right(z)):
            self._rotate(y)
            return y
        else:
            self._rotate(x)    
            self._rotate(x)
            return x
